# Harmony PI 20.2 Demo

This notebook provides condensed examples of using Harmony to perform specific features that were built for PI 20.2.  For more a general introduction and tutorial, see [Harmony API Introduction](./Harmony%20Api%20Introduction.ipynb).  Useful helpers for making the calls found in this note book can be found under the [docs/notebook-helpers](./notebook-helpers) folder

## Prerequisites

1. Install Python 3.  This notebook is tested to work in 3.8 but should work in most recent 3.x versions.
2. Install Jupyter: `pip install jupyterlab`
3. Setup your `~/.netrc` for Earthdata Login as described in [Harmony API Introduction](./Harmony%20Api%20Introduction.ipynb)
4. Run the following cell to install Python dependencies, import necessary modules, and set notebook defaults

In [ ]:
%load_ext autoreload
%autoreload

import sys, os
# Install dependencies into the Jupyter Kernel
!{sys.executable} -m pip install -q -r ../notebook_helpers/requirements.txt
%matplotlib inline

# Import libraries used throughout the notebook
sys.path.append(os.path.normpath(os.path.join(os.getcwd(), '..')))
from notebook_helpers import get, post, show, get_data_urls, show_async, show_async_condensed, print_async_status, show_shape

## ASF Data Transformations in Harmony's Production Environment

ASF's Sentinel-1 Interferograms (BETA) product is wired up to Harmony's new production environment in preparation for going operational next PI (don't tell the world yet though!)

In [ ]:
asf_collection = 'C1595422627-ASF'
coverages_root = 'https://harmony.earthdata.nasa.gov/{collection}/ogc-api-coverages/1.0.0/collections/{variable}/coverage/rangeset'

### Variable and spatial subsetting with projecting, reformtatting output to PNG and spatial constraints
Each parent NetCDF is approx. 60 MB and the subsetted pngs and geotiffs are well under 1 MB each.

In [ ]:
response = get(
    coverages_root.format(
        collection=asf_collection, 
        variable='%2Fscience%2Fgrids%2Fdata%2FunwrappedPhase'), 
    params={
       # 'granuleID': ['G1715962900-ASF', 'G1715965586-ASF'],
        'format': 'image/png',
        'outputcrs': 'EPSG:2230',
        'subset': [
            'lon(-115.5:-115.25)', 
            'lat(33:33.1)',
             'time("2020-03-12T00:00:00Z":"2020-03-13T03:00:00Z")'
            ]})
show_async_condensed(response)

## Job Management and Scaling

### Listing of Jobs
If you are in the operators group, you can see all jobs in the system at https://harmony.earthdata.nasa.gov/admin/jobs.
Users can see their own jobs at https://harmony.earthdata.nasa.gov/jobs.

In [ ]:
admin_jobs='https://harmony.earthdata.nasa.gov/admin/jobs'
my_jobs='https://harmony.earthdata.nasa.gov/jobs'
response = get(admin_jobs)
print("Jobs viewable as admin:", response.json()['count'])

response = get(my_jobs)
print("Jobs viewable as myself:", response.json()['count'])

### Queing of jobs
To show that jobs are queuing and to have time to cancel them, load the system up with a few longer running jobs

In [ ]:
#Request 1 - Spatial and variable subset; will find 654 granules and process the first 20
response1 = get(
    coverages_root.format(collection=asf_collection, variable='%2Fscience%2Fgrids%2Fdata%2Famplitude'), 
    params={
        'format': 'image/png',
        'outputcrs': 'EPSG:2230',
        'subset': ['lon(-115.5:-115.25)', 'lat(33:33.1)']})

#Request 2 - Reformat of Harmony test product
response2 = get(coverages_root.format(collection='C1756916832-XYZ_PROV', variable='all'), params={'format': 'image/png'})

#Request 3 - Reformat of Harmony test product
response3 = get(coverages_root.format(collection='C1756916832-XYZ_PROV', variable='all'), params={'format': 'image/tiff'})
   

### List Recent jobs
The jobs endpoints support paging to make it easier to navigate.

In [ ]:
response = get(my_jobs,
    params={'page': '1','limit': '10'})

body = response.json()

for job in body['jobs']:
    print_async_status(job)


#### Cancel a job

In [ ]:


my_jobs_cancel_root=my_jobs+'/{job_id}/cancel'
response = post(my_jobs_cancel_root.format(job_id='5e03edf3-3e2f-4c77-87b4-a5fd8ff230ff'))

print_async_status(response.json())